# 강좌 엑셀 파일에서 불러오기

In [15]:
# from urllib.request import urlopen
# html = urlopen("https://www.frip.co.kr/daily/learning/all?orderBy=popular&page=1")  
# bsObject = BeautifulSoup(html, "html.parser")
# print(bsObject)

In [39]:
import requests
from bs4 import BeautifulSoup

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
url = "https://www.frip.co.kr/products/108923"
webpage = requests.get(url, headers=headers)
soup = BeautifulSoup(webpage.content, "html.parser")
soup.select(
'body')
# print(soup)

[<body><noscript>If you're seeing this message, that means <strong>JavaScript has been disabled on your browser</strong>, please <strong>enable JS</strong> to make this app work.</noscript><div id="app"></div><div id="modal-root"></div><div id="dnd-root"></div><link href="https://fonts.googleapis.com/css?family=Noto+Sans+KR:300,400,500,700" rel="stylesheet"/><script src="/runtime~main.bcf9f3a8eff7fa7c2423.js" type="text/javascript"></script><script src="/vendors~main.3749da4eac8b61e1a652.chunk.js" type="text/javascript"></script><script src="/main.680bb431ba4e89ad202b.chunk.js" type="text/javascript"></script></body>]

In [8]:
with open('lecture.csv') as file:
    csv_data = []
    for line in file.readlines():
        csv_data.append(line.split(','))
print(csv_data[0][0])

강좌명


In [ ]:
import numpy
csv_data = numpy.loadtxt('lecture.csv', delimiter=',')


In [ ]:
import pandas as pd
csv_data = pd.read_csv("./lecture.csv")
print(csv_data)

# 셀리니움을 활용하여 frip에서 크롤링

In [42]:
pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge selenium

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\sehyu\anaconda3

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    selenium-3.141.0           |py37h8055547_1001         937 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m
  selenium           conda-forge/win-64::selenium-3.141.0-py37h8055547_1001

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.2-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1




p



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




In [1]:
import time
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime

driver = webdriver.Chrome('./chromedriver')

## 페이지 넘어가면서 크롤링

In [2]:
links = []
for i in range(1, 179):
    driver.get('https://www.frip.co.kr/daily/learning/all?page='+ str(i))
    time.sleep(1)
    lectures = driver.find_elements_by_class_name("sc-kGXeez")
    for lecture in lectures:
        a_name = lecture.find_element_by_tag_name('a').get_attribute('href')
        links.append(a_name)

# 크롤링한 페이지 url csv파일로 저장

## DataFrame 방식으로 저장

In [3]:
import pandas as pd

dataframe = pd.DataFrame(links)
dataframe.to_csv('./frip.csv', header=False,index=False)

## csv writer 방식으로 저장

In [4]:
import csv

In [150]:
f = open('./frip_csv.csv', 'w', newline='')
wr = csv.writer(f)
for link in links:
    wr.writerow([link])
f.close()

# 페이지 url이 저장된 csv 파일 불러오기

In [8]:
f = open('./frip.csv', 'r')
rdr = csv.reader(f)
new_links = []
for line in rdr:
    new_links.append(line[0])
f.close()

In [151]:
f = open('./frip_csv.csv', 'r')
rdr = csv.reader(f)
new_links = []
for line in rdr:
    new_links.append(line[0])
f.close()

In [86]:
new_links[1]

'https://www.frip.co.kr/products/108923'

# 페이지 별로 돌아가면서 데이터 크롤링

## 한개만 테스트

In [9]:
driver.get("https://www.frip.co.kr/products/114211")
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
meeting_id = "https://www.frip.co.kr/products/114211".split('/')[-1]
uid = driver.find_element_by_class_name('nCgAM').find_element_by_tag_name('a').get_attribute('href')
writer = soup.select_one('span.gYdgbL')
created_at = datetime.now()
updated_at = datetime.now()
is_period = 0
meeting_date = datetime.now()
temp = soup.select('section.sc-dUcZlc > div.sc-ZUflv > div.sc-esoVGF')
if temp[1].text[0].isdigit():
    temp_date = ''
    for t in temp[1].text:
        if t == '외':
            break
        if t in '(월화수목금토일요일년)':
            continue
        if t == ' ':
            t = '-'
        temp_date += t
    temp_date = temp_date[:-1]
    meeting_date = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
period_date = ''
is_class = 1
max_person = 20
now_person = 1
content = '[내용]' + soup.select('div.cSNowP')[0].text
ref_url = new_links[0]
address = soup.select_one('div.sc-kRCAcj').text[:-5]
fee = soup.select_one('div.sc-eBipZS').text[:-1].replace(',', '')
unit = '원'
is_active = 1
like_temp = ''
for t in temp[0].text:
    if not t.isdigit():
        break
    like_temp += t
like_cnt = int(like_temp)
view_cnt = 0
score = 0
if temp[3].text[6].isdigit():
    score = float(temp[3].text[6:])
main_category = 0
tags = ''

IndexError: string index out of range

In [175]:
meetings.append(
    [
        meeting_id,
        uid,
        writer,
        created_at,
        updated_at,
        is_period,
        meeting_date,
        period_date,
        is_class,
        max_person,
        now_person,
        content,
        ref_url,
        address,
        fee,
        unit,
        is_active,
        like_cnt,
        view_cnt,
        score,
        main_category,
        tags
    ]
)

## 미팅 시간 연산

In [85]:
from datetime import datetime

temp_date = ''
for t in temp[1].text:
    if t == '외':
        break
    if t in '(월화수목금토일요일년)':
        continue
    if t == ' ':
        t = '-'
    temp_date += t
temp_date = temp_date[:-1]
datetime_object = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
print(datetime_object)

2020-05-23 13:00:00


## columns 생성

In [10]:
import pandas as pd
from datetime import datetime

meeting_columns = (
    "meeting_id",
    "uid",
    "writer",
    "created_at",
    "updated_at",
    "is_period",
    "meeting_date",
    "period_date",
    "is_class",
    "max_person",
    "now_person",
    "content",
    "ref_url",
    "address",
    "fee",
    "unit",
    "is_active",
    "like_cnt",
    "view_cnt",
    "score",
    "main_category",
    "tags"
)

image_columns = (
    "meeting_id",
    "image_url"
)

## 제대로 테스트

In [11]:
meetings = []
images = []

In [20]:
for link in range(350, len(new_links)):
    print(new_links[link])
    driver.get(new_links[link])
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    meeting_id = new_links[link].split('/')[-1]
    uid = driver.find_element_by_class_name('nCgAM').find_element_by_tag_name('a').get_attribute('href')
    writer = soup.select_one('span.gYdgbL')
    created_at = datetime.now()
    updated_at = datetime.now()
    is_period = 0
    meeting_date = datetime.now()
    temp = soup.select('section.sc-dUcZlc > div.sc-ZUflv > div.sc-esoVGF')
    if temp[-3].text == '':
        continue
    elif temp[-3].text[0].isdigit():
        temp_date = ''
        for t in temp[1].text:
            if t == '외':
                break
            if t in '(월화수목금토일요일년)':
                continue
            if t == ' ':
                t = '-'
            temp_date += t
        temp_date = temp_date[:-1]
        try:
            meeting_date = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
        except:
            meeting_date = datetime.now()
    period_date = ''
    is_class = 1
    max_person = 20
    now_person = 1
    content = '[내용]' + soup.select('div.cSNowP')[0].text
    ref_url = new_links[link]
    print(soup.select_one('div.sc-kRCAcj').text[:-5])
    address = soup.select_one('div.sc-kRCAcj').text[:-5]
    fee = soup.select_one('div.sc-eBipZS').text[:-1].replace(',', '')
    unit = '원'
    is_active = 1
    like_temp = ''
    for t in temp[0].text:
        if not t.isdigit():
            break
        like_temp += t
    try:
        like_cnt = int(like_temp)
    except:
        like_cnt = 0
    view_cnt = 0
    score = 0
    if temp[-1].text[6].isdigit():
        score = float(temp[-1].text[6:])
    main_category = 0
    tags = ''
    
    # 이미지 저장
    images_link = driver.find_elements_by_class_name('cljRKC')
    image_urls = set()
    for i in images_link:
        image_urls.add(str(i.get_attribute('src')))
    image_url = ' '.join(image_urls)
    
    # 데이터 저장
    meetings.append(
    [
        meeting_id,
        uid,
        writer,
        created_at,
        updated_at,
        is_period,
        meeting_date,
        period_date,
        is_class,
        max_person,
        now_person,
        content,
        ref_url,
        address,
        fee,
        unit,
        is_active,
        like_cnt,
        view_cnt,
        score,
        main_category,
        tags
    ]
    )
        
    images.append(
    [
        meeting_id,
        image_url
    ]
    )


https://www.frip.co.kr/products/136215
온라인
https://www.frip.co.kr/products/122031
서울 서대문구 이화여대길 7, 월성빌딩 3층
https://www.frip.co.kr/products/121139
서울시 마포구 토정로25길 31, 1층 리브모아 도예공방
https://www.frip.co.kr/products/121301
서울시 광진구 능동로 41길 42 1층 슬로우 허그
https://www.frip.co.kr/products/110491
라슘바(강남구 역삼동 830-54 B1, 강남역 4번 출구에서 도보 8분)
https://www.frip.co.kr/products/136026
강원도 고성군 죽왕면 오호천길 23
https://www.frip.co.kr/products/135549
서울시 은평구 가좌로 247, 2층
https://www.frip.co.kr/products/132899
경기도 수원시 영통구 에듀타운로 101, 에듀하임1309오피스텔 105동 B103호
https://www.frip.co.kr/products/111755
서울시 서초구 신반포로23길 41, 잠원쇼핑센타 3층 314호
https://www.frip.co.kr/products/136177
서울 마포구 서교동 457-1 1층 봉봉비
https://www.frip.co.kr/products/133147
경기도 수원시 영통구 영통동 989-2 영통역 3번 출구 1분 카페 나뇽 (현대아파트 7단지내 상가 태안농협 건물 1층102호 나뇽)
https://www.frip.co.kr/products/120203
강원도 강릉시 옥계면 헌화로 221-4
https://www.frip.co.kr/products/129393
서울시 중구 삼일대로 358 1층 스타벅스
https://www.frip.co.kr/products/136274
서울시 중랑구 동일로 775 (중화3동 그린프라자 B1) 중화스쿼시헬스클럽
https://www.f

서울시 송파구 송파동 108-10번지 1
https://www.frip.co.kr/products/113249
달드림 (서울 양천구 남부순환로 604-1 지하1층)
https://www.frip.co.kr/products/119304
서울시 강남구 삼성동 151-29, 이호빌딩 2층
https://www.frip.co.kr/products/116326
권영세 클라이밍센터 (구. 헥사 클라이밍센터) (3호선 충무로역 1번 출구에서 도보로 약 10분 거리), 서울특별시 중구 필동3가 28-19
https://www.frip.co.kr/products/124343
서울특별시 강남구 강남대로96길 12, 스파앤뷰티
https://www.frip.co.kr/products/135388
서울시 서초구 양재동 3-6 B1층 '나무워크샵'
https://www.frip.co.kr/products/135555
서울시 중구 을지로5길 19 111
https://www.frip.co.kr/products/135355
서울시 용산구 용산동 2가 30번지 1층 Breezy
https://www.frip.co.kr/products/135365
경기도 성남시 분당구 느티로69번길 15, 작업실보통날
https://www.frip.co.kr/products/134788
매주 월, 수, 금요일에 메일을 보내드려요.
https://www.frip.co.kr/products/132198
서울시 구로구 개봉로17라길 50, 미소공방(제로플러스)
https://www.frip.co.kr/products/130638
경기도 성남시 분당구 황새울로116번길 16, 1층 버블다움
https://www.frip.co.kr/products/111258
서울특별시 마포구 독막로 134-3, 2층 (광흥창역(6호선) 6번 출구 도보 5분 이내 거리)
https://www.frip.co.kr/products/117383
서울시 관악구 조원로2길21, 문화살롱 501호
https://www.frip.co.kr/p

서울시 송파구 방이동 62-12, 4층
https://www.frip.co.kr/products/129147
경기도 용인시 기흥구 보정로 117, 리베로3 (1층) 145호 mn/E.d.t
https://www.frip.co.kr/products/122212
서울시 성동구 연무장길 9-6, 건영빌딩 3F 링스
https://www.frip.co.kr/products/116297
두클라이밍짐(부산광역시 남구 대연3동 60-13 황금빌딩 2층)
https://www.frip.co.kr/products/110054
서울시 성동구 성수이로 87, 성문빌딩 1층
https://www.frip.co.kr/products/117401
서울시 중구 장충단로7길 11번지 1층
https://www.frip.co.kr/products/135797
경기도 수원시 영통구 에듀타운로 101, 에듀하임 105동 B103호
https://www.frip.co.kr/products/135330
서울 마포구 방울내로 80-16 1층 지아네요리공작소
https://www.frip.co.kr/products/135051
서울 강남구 학동로 38길 25, 지하1층 에이원민턴
https://www.frip.co.kr/products/134458
경기 부천시 송내대로39번길 41 1층
https://www.frip.co.kr/products/133635
서울시 금천구 탑골로 47 1층 원바이올린
https://www.frip.co.kr/products/130754
경기도 성남시 분당구 황새울로116번길 16, 1층 버블다움
https://www.frip.co.kr/products/128336
서울시 마포구 신촌로24길 17, 2층 화실 그리다
https://www.frip.co.kr/products/120962
서울시 광진구 자양동 704-1, 뚝섬 윈드서핑장 34호 하이윈드클럽
https://www.frip.co.kr/products/118175
서울시 영등포구 문래동2가 14-91 일럼
http

서울 서초구 양재동 286-12 헤이미쉬 케이크
https://www.frip.co.kr/products/116324
대구광역시 중구 중앙대로 412-26 3층
https://www.frip.co.kr/products/122767
서울시 광진구 능동로41길 42, 1층 슬로우허그
https://www.frip.co.kr/products/132312
서울시 관악구 남부순환로 1919, B1층 비올즈 플라워카페
https://www.frip.co.kr/products/115529
Cafe Jool (서울시 마포구 동교동 181-6 1F)
https://www.frip.co.kr/products/135839
서울 성동구 성수일로10길 26 하우스디세종타워 2층 204호 나의근육사용설명서
https://www.frip.co.kr/products/136115
광주시 동구 장동로 7, 1층 르뱅
https://www.frip.co.kr/products/135083
서울 강남구 테헤란로53길 28 지하2층, 지하3층
https://www.frip.co.kr/products/135459
서울시 강서구 공항대로46길 25 3층 빅픽처컴퍼니
https://www.frip.co.kr/products/135441
온라인
https://www.frip.co.kr/products/125340
서울시 용산구 회나무로 26, 1층 프레그릿
https://www.frip.co.kr/products/133786
서울시 서대문구 세무서길 85 2층
https://www.frip.co.kr/products/134692
강원도 강릉시 옥계면 헌화로 237, 서프소울
https://www.frip.co.kr/products/132886
서울특별시 마포구 합정동 431-42, B1 실버라스튜디오
https://www.frip.co.kr/products/134341
경기 부천시 송내대로39번길 41, 1층
https://www.frip.co.kr/products/134271
제주특별자치도 서귀포시 성산

서울시 강남구 논현로26길 25 (1층 그리다블룸)
https://www.frip.co.kr/products/111869
서울시 강남구 도곡동 964 상가동 101호 SVET
https://www.frip.co.kr/products/116406
서울시 성북구 동소문로 17길 29-2, 동소문피스커피 / 경기도 의정부시 의정로46번길 18 1층, 커피클래스
https://www.frip.co.kr/products/115568
서울시 마포구 망원동 384-19번지 201호 그린나무공방
https://www.frip.co.kr/products/116460
서울시 마포구 신촌역 6번 출구 (자세한 위치는 구매후 문자안내)
https://www.frip.co.kr/products/120252
서울시 양천구 목동동로 53 (상세 주소 추후 공지)
https://www.frip.co.kr/products/113810
강남역 인근 스터디룸 (신청하신 분에 한해 신청 마감 후 안내드림)
https://www.frip.co.kr/products/120929
서울 영등포구 가마산로61나길 11 1층 백스트로크
https://www.frip.co.kr/products/109373
서울시 관악구 낙성대역 3번 출구 동림오피스텔 b3
https://www.frip.co.kr/products/115452
제주도 서귀포시 부두로 51
https://www.frip.co.kr/products/136037
경기도 시흥시 대은로 16-4, 2층 카페 천하
https://www.frip.co.kr/products/135964
경기도 시흥시 하중로231번길 11-1, 1층 카페와이 (교육식)
https://www.frip.co.kr/products/136137
온라인
https://www.frip.co.kr/products/136087
경기 부천시 춘의동 178-32 1층 마노드
https://www.frip.co.kr/products/135249
우리 집!
https://www.frip.co.k

...

수원시 영통구 망포동11-9 다부타운 1층 드로잉에세이화시
https://www.frip.co.kr/products/109086
서울시 관악구 봉천로 607, 2층 cowhide
https://www.frip.co.kr/products/115265
서울시 용산구 동자동 24-32
https://www.frip.co.kr/products/111567
경기도 성남시 분당구 느티로69번길 15, 작업실보통날
https://www.frip.co.kr/products/109060
서울시 서초구 양재동 364번지 1층
https://www.frip.co.kr/products/94892
서울시 서초구 반포동 107-26 풀하우스 1층 마리에르플라워
https://www.frip.co.kr/products/116504
서울시 강서구 화곡동 332-27번지 1층 nayuna
https://www.frip.co.kr/products/115959
엄구스 가죽공방 (서울시 동작구 동작동 333번지, 이스힐스테이트 상가 1층, 130호 )
https://www.frip.co.kr/products/117389
서울시 강남구 강남대로94길 18, 삼행빌딩 2층
https://www.frip.co.kr/products/116163
서울특별시 용산구 한남동 28-12, 우석빌딩 401호
https://www.frip.co.kr/products/115566
은정빌딩 5층 (서울시 서초구 양재동 15-22)
https://www.frip.co.kr/products/115772
서울시 강동구 암사동 504-29번지 지하1층
https://www.frip.co.kr/products/115759
서울시 송파구 올림픽로 342 아울타워 4F
https://www.frip.co.kr/products/116555
서울시 서초구 강남대로 30길 12 은정빌딩 5층
https://www.frip.co.kr/products/109064
서울시 서초구 양재동 364번지 1층 스튜디오벨
https://www.fr

서울시 금천구 범안로 1142, 하우스디 더 스카이밸리 가산2차 9층 901호
https://www.frip.co.kr/products/132329
서울시 강남구 대치동 974, 상가동1층 111호
https://www.frip.co.kr/products/132199
서울시 강남구 테헤란로 322 23층
https://www.frip.co.kr/products/132244
서울시 성동구 금호로 100, 하이피아프라자 602-2호
https://www.frip.co.kr/products/120826
서울 광진구 화양동 33-30 드래곤플라이 연습실 B1
https://www.frip.co.kr/products/130838
서울시 서초구 서초대로 78길 44 502호  공간이즈24시스터디룸&스터디까페
https://www.frip.co.kr/products/132140
서울시 강남구 역삼로 220 1층 캐컴
https://www.frip.co.kr/products/131986
서울시 성북구 성북로8길 4, 4층
https://www.frip.co.kr/products/121266
서울시 양천구 공항대로 532, 등촌역 5번출구 신한은행건물 지하1층
https://www.frip.co.kr/products/131756
경기도 성남시 분당구 황새울로116번길 16, 1층 버블다움
https://www.frip.co.kr/products/131439
인천시 미추홀구 관교동 494-5, 1층 로즐라인 비누가게
https://www.frip.co.kr/products/131364
온라인(슬랙 메신저 사용)
https://www.frip.co.kr/products/111595
서울시 강남구 압구정로18길 25, 2층
https://www.frip.co.kr/products/130991
서울시 서초구 논현로 157-6, 삼신빌딩 지하
https://www.frip.co.kr/products/131054
서울시 강남구 역삼동 657-12, 화향
https://www.frip.c

경상북도 포항시 북구 두호동 대곡로20 4
https://www.frip.co.kr/products/115797
안양시 만안구 장내로137번지 5층 청아트룸
https://www.frip.co.kr/products/115764
서울시 서초구 방배동 471-13번지 1층
https://www.frip.co.kr/products/111619
서울시 중구 충무로4길 19-4 (근처 도착하시면 전화 부탁드립니다^^)
https://www.frip.co.kr/products/115944
부산 동래구 안연로 56-1 3층 사진관희안
https://www.frip.co.kr/products/116115
인천지하철 선학역 4번 출구
https://www.frip.co.kr/products/115974
경기도 화성시 동탄오산로 86-11, 11층
https://www.frip.co.kr/products/115631
서강대 야외 농구코트 (서울시 마포구 대흥동 22-60)
https://www.frip.co.kr/products/115673
온라인
https://www.frip.co.kr/products/115424
혜화역 카페 M
https://www.frip.co.kr/products/115327
서울특별시 송파구 송파대로 39길 4, B1
https://www.frip.co.kr/products/115927
송도잠수풀, 성남잠수풀, 일산 수작스튜디어 중 선택 가능
https://www.frip.co.kr/products/116179
부산시 해운대구 마린시티2로 제니스스퀘어 2층 236호 엘리아가죽공방
https://www.frip.co.kr/products/115341
서울시 강서구 등촌3동 주공 5단지 502
https://www.frip.co.kr/products/115817
경기도 용인시 기흥구 동백동 631-8번지 102호 슈룩아트공방
https://www.frip.co.kr/products/115573
부산시 부산진구 양지로 54, 동의과학대학 잠수풀장
https

서울시 마포구 서교동442-1 성광빌딩201호
https://www.frip.co.kr/products/108696
서울 송파구 송파대로 458 해광빌딩 204호 아트블라썸
https://www.frip.co.kr/products/130760
경기 고양시 덕양구 화신로272번길 29, 307호
https://www.frip.co.kr/products/127038
서울시 광진구 아차산로 552, 2-705
https://www.frip.co.kr/products/114147
신촌 위지안카페( 서울마포구 고산길 6 위지안빌딩)
https://www.frip.co.kr/products/116541
민미아 주얼리 아뜰리에 ( 서울시 강남구 신사동 580, 지하 1층)
https://www.frip.co.kr/products/131627
경기 부천시 송내대로 14 상가 2층
https://www.frip.co.kr/products/127063
전 지역
https://www.frip.co.kr/products/126966
서울시 강남구 대치동 974, 상가동1층 111호
https://www.frip.co.kr/products/117425
서울특별시 강남구 논현동 9-18, 4층
https://www.frip.co.kr/products/123850
서울시 강서구 월정로 186-1, 지층 고페르퍼니쳐
https://www.frip.co.kr/products/127805
서울 중구 장충단로7길 11 1층 라라디저트
https://www.frip.co.kr/products/109061
서울시 서초구 양재동 364번지 1층
https://www.frip.co.kr/products/116433
서울시 마포구 합정동 367-8 지층 1호
https://www.frip.co.kr/products/134403
배송상품
https://www.frip.co.kr/products/132585
서울시 용산구 동자동 24-32, 1층 온애어 아뜰리에
https://www.frip.co.kr/p

IndexError: list index out of range

In [21]:
meeting_frame = pd.DataFrame(data=meetings, columns=meeting_columns)

In [22]:
meeting_frame.to_csv('./meeting_data.csv', header=True,index=False, encoding='utf-8')

In [23]:
meeting_frame.tail(100)

,meeting_id,uid,writer,created_at,updated_at,is_period,meeting_date,period_date,is_class,max_person,...,ref_url,address,fee,unit,is_active,like_cnt,view_cnt,score,main_category,tags
2170,120693,https://www.frip.co.kr/hosts/7552,[썸머브리즈],2020-05-25 03:11:46.317376,2020-05-25 03:11:46.317376,0,2020-05-25 03:11:46.317376,,1,20,...,https://www.frip.co.kr/products/120693,"서울시 마포구 백범로24길 9, 1층",50000,원,1,318,0,4.89,0,
2171,115595,https://www.frip.co.kr/hosts/968,[다미],2020-05-25 03:11:53.236959,2020-05-25 03:11:53.236959,0,2020-05-25 03:11:53.236959,,1,20,...,https://www.frip.co.kr/products/115595,서울시 강동구 암사동 504-29번지 지하 1층,45000,원,1,956,0,4.89,0,
2172,123411,https://www.frip.co.kr/hosts/7983,[센트오브블루밍],2020-05-25 03:11:55.964634,2020-05-25 03:11:55.964634,0,2020-05-25 03:11:55.964634,,1,20,...,https://www.frip.co.kr/products/123411,"서울시 강남구 논현동 172-22, 1층",40000,원,1,265,0,4.88,0,
2173,117457,https://www.frip.co.kr/hosts/6753,[리우],2020-05-25 03:11:59.225880,2020-05-25 03:11:59.225880,0,2020-05-25 03:11:59.225880,,1,20,...,https://www.frip.co.kr/products/117457,서울시 강북구 미아동 309-44 (1층 리우캔들),20000,원,1,120,0,4.88,0,
2174,116154,https://www.frip.co.kr/hosts/4718,[감성소녀],2020-05-25 03:12:02.317610,2020-05-25 03:12:02.317610,0,2020-05-25 03:12:02.317610,,1,20,...,https://www.frip.co.kr/products/116154,서울시 서초구 방배동 908-26 지하 일층,50000,원,1,87,0,4.88,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,131764,https://www.frip.co.kr/hosts/9765,[달콤베이킹],2020-05-25 03:18:12.580341,2020-05-25 03:18:12.580341,0,2020-05-25 03:18:12.580341,,1,20,...,https://www.frip.co.kr/products/131764,"서울시 마포구 백범로 152, 공덕파크자이 201동1층",100000,원,1,4,0,0.00,0,
2266,133550,https://www.frip.co.kr/hosts/10843,[verse],2020-05-25 03:18:17.484602,2020-05-25 03:18:17.484602,0,2020-05-25 03:18:17.484602,,1,20,...,https://www.frip.co.kr/products/133550,"부산시 금정구 기찰로 16, 1층 verse",50000,원,1,9,0,0.00,0,
2267,133572,https://www.frip.co.kr/hosts/10852,[다올플라워],2020-05-25 03:18:21.625259,2020-05-25 03:18:21.625259,0,2020-05-25 03:18:21.625259,,1,20,...,https://www.frip.co.kr/products/133572,"서울시 서초구 서초대로33길 55, 1층 다올플라워",60000,원,1,6,0,0.00,0,
2268,133577,https://www.frip.co.kr/hosts/2871,[국가대표 통역사],2020-05-25 03:18:24.844966,2020-05-25 03:18:24.844966,0,2020-05-25 03:18:24.844966,,1,20,...,https://www.frip.co.kr/products/133577,온라인 실시간 수업입니다 (Zoom 화상 회의 프로그램),45000,원,1,8,0,0.00,0,


In [24]:
image_frame = pd.DataFrame(data=images, columns=image_columns)